__Disclaimer__: Welcome to the Emotion Tracker developed in Collaboration with 
Prof. Peter Gloor and the support from Josephine Van Delden. This is an alpha version of the upcoming web service.

### Importing Modules

In [13]:
# General
import time
import sys
import io
import os
import base64

# Data Processing
import cv2
import numpy as np
import pandas as pd
import subprocess
from scipy.signal import chirp, find_peaks, peak_widths
import librosa
import librosa.display
# Thresholding Testing
from skimage.data import page
from skimage.filters import (threshold_otsu, threshold_niblack, threshold_sauvola)


# Visual
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display
from IPython.display import clear_output
from datetime import datetime
import altair as alt
# import matplotlib.pyplot as plt

# Google Colab
from google.colab import files
from google.colab.patches import cv2_imshow

In [14]:
# Clean Up Temporary Movie Files
try:
    os.remove("pls_delete.mp4")
    os.remove("pls_delete.mpeg")
    os.remove("pls_delete.wav")
    os.remove("pls_delete_play.mp4")
except FileNotFoundError:
    print("Downloading Videos...")
else:
    print("RM. Downloading Videos...")

# To Do: Check if files are already downloaded, else download them.

# # Downloading
# # Videos

# !curl -L https://www.dropbox.com/s/yk0es4189d9bc6s/plants_alternating_emotions_w_60s_silence%2Bday.mov?dl=1 -o alternating_60s_new.mov -s
# !curl -L https://www.dropbox.com/s/yncd1m5b462qxbk/plants_alternating_emotions_w_10s_silence.mov?dl=1 -o alternating_10s.mov -s
# !curl -L https://www.dropbox.com/s/dvj63tdba0ib2xk/plants_alternating_emotions_w_60s_silence.mov?dl=1 -o alternating_60s.mov -s
# !curl -L https://www.dropbox.com/s/yrlxynmnr9aqvsf/paprika_lq.mov?dl=1 -o paprika.mov -s
# !curl -L https://www.dropbox.com/s/j5a18aw661qz8is/pot_plastic.mov?dl=1 -o pot.mov -s
# !curl -L https://www.dropbox.com/s/1o60nbvionp4uu0/mimosa_nosound-400x.mp4?dl=1 -o mimosa.mov -s
!curl -L https://www.dropbox.com/s/dun532o3s1c49xi/Codariocalyx%2BMimosa-400x.mp4?dl=1 -o mimosa_400x.mov -s
!curl -L https://www.dropbox.com/s/lb24eyw57qdiocc/codariocalyx_nopeter_nosound-400x.mp4?dl=1 -o codariocalyx_no_peter.mov -s
# !curl -L https://www.dropbox.com/s/1o60nbvionp4uu0/mimosa_nosound-400x.mp4?dl=1 -o mimosa_400x_no_sound.mov -s
# # # Images
# !curl -o happy.png -L https://i.imgur.com/PXpWO5Cs.png -s

# !curl -o sad.png -L https://i.imgur.com/mAYh4Qts.png -s

In [15]:
#@title # Parameters, Constants

Video = "codariocalyx_no_peter.mov"  #@param ['paprika.mov', 'pot.mov', 'mimosa.mov', 'mimosa_400x.mov', 'codariocalyx_no_peter.mov', 'mimosa_400x_no_sound.mov']
INPUT_VIDEO = Video

# Constants
INPUT_VIDEO_NAME = 'pls_delete'
OUTPUT_VIDEO_NAME_MPEG = 'pls_delete.mpeg'
OUTPUT_VIDEO_NAME_MP4 = '{}.mp4'.format(INPUT_VIDEO_NAME)
OUTPUT_VIDEO_NAME_WAV = '{}.wav'.format(INPUT_VIDEO_NAME)
FRAMES_PER_SECOND = 30
x = 0 
y = 0 
w = 0
h = 0

# Params
Shrink_Regions_by_px =  12#@param {type:"integer"}
ERODE_1 =  Shrink_Regions_by_px
ERODE_2 = ERODE_1 

# Dilate: Makes intersting regions grow.
Grow_Regions_by_px =  17#@param {type:"integer"}
DILATE_1 = Grow_Regions_by_px
DILATE_2 = DILATE_1

# Lumen_Threshold = 250#@param {type:"integer"} 
# MASK_THRESH = Lumen_Threshold
Countour_Approximation = 60#@param {type:"integer"}
CONTOUR_AREA = Countour_Approximation

In [16]:
# @title # Leaf Class hide

class Leaf():
    def __init__(self, id, hsv_frame, track_window):

        self.id = id

        self.track_window = track_window
        self.term_crit = \
            (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 10, 1)

        # Initialize the histogram.
        self.x, self.y, self.w, self.h = track_window
        roi = hsv_frame[y:y+h, x:x+w]
        roi_hist = cv2.calcHist([roi], [0], None, [16], [0, 180])
        self.roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255,
                                      cv2.NORM_MINMAX)

        # Initialize the Kalman filter.
        self.kalman = cv2.KalmanFilter(4, 2)
        self.kalman.measurementMatrix = np.array(
            [[1, 0, 0, 0],
             [0, 1, 0, 0]], np.float32)
        self.kalman.transitionMatrix = np.array(
            [[1, 0, 1, 0],
             [0, 1, 0, 1],
             [0, 0, 1, 0],
             [0, 0, 0, 1]], np.float32)
        self.kalman.processNoiseCov = np.array(
            [[1, 0, 0, 0],
             [0, 1, 0, 0],
             [0, 0, 1, 0],
             [0, 0, 0, 1]], np.float32) * 0.03
        cx = x+w/2
        cy = y+h/2
        self.kalman.statePre = np.array(
            [[cx], [cy], [0], [0]], np.float32)
        self.kalman.statePost = np.array(
            [[cx], [cy], [0], [0]], np.float32)
        
    def get_plant_values(self): 
        return np.array([ self.id, self.x, self.y, self.w, self.h ])
    
    def update(self, frame, hsv_frame):

        back_proj = cv2.calcBackProject(
            [hsv_frame], [0], self.roi_hist, [0, 180], 1)

        ret, self.track_window = cv2.meanShift(
            back_proj, self.track_window, self.term_crit)
        x, y, w, h = self.track_window
        center = np.array([x+w/2, y+h/2], np.float32)

        prediction = self.kalman.predict()
        estimate = self.kalman.correct(center)
        center_offset = estimate[:,0][:2] - center
        self.track_window = (x + int(center_offset[0]),
                             y + int(center_offset[1]), w, h)
        x, y, w, h = self.track_window

        # Draw the predicted center position as a blue circle.
        cv2.circle(frame, (int(prediction[0]), int(prediction[1])),
                   4, (255, 0, 0), -1)

        # Draw the corrected tracking window as a cyan rectangle.
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 255, 0), 2)

        # Draw the ID above the rectangle in blue text.
        cv2.putText(frame, 'ID: %d' % self.id, (x, y-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0),
                    1, cv2.LINE_AA)
        

# Detection


In [ ]:
# @title ## Playground

# # Object Detection different Thresholding Algorithms

# # Load Video
# cap = cv2.VideoCapture('mimosa_400x.mov')
# arr = []
# no_of_frame = 0
# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))

# erode_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ERODE_1, ERODE_2))
# dilate_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (DILATE_1, DILATE_2))
# num_history_frames_populated = 0

# fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
# out = cv2.VideoWriter(OUTPUT_VIDEO_NAME_MPEG, fourcc, FRAMES_PER_SECOND, (frame_width, frame_height))
# grabbed_frames = 0

# # ---------Thresholding------------

# grabbed, frame = cap.read()
# frame = frame.copy()
# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# # Binary Thresholding
# _, thresh = cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)

# # Adaptive Thresholding
# thresh = cv2.adaptiveThreshold(frame, 120, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# # Niblack and Sauvola Thresholding
# binary_global = frame > threshold_otsu(frame)

# window_size = 25
# thresh_niblack = threshold_niblack(frame, window_size=window_size, k=0.8)
# thresh_sauvola = threshold_sauvola(frame, window_size=window_size)

# binary_niblack = frame > thresh_niblack
# cv2_imshow(binary_niblack)
# binary_sauvola = frame > thresh_sauvola
# cv2_imshow(binary_sauvola)

# # Find intersting points
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# corners = cv2.goodFeaturesToTrack(gray,25,0.01,10)
# corners = np.int0(corners)

# for i in corners:
#     x,y = i.ravel()
#     cv2.circle(gray,(x,y),3,255,-1)
# # cv2_imshow(gray)

# # Morphology
# cv2.erode(thresh, erode_kernel, thresh, iterations=2)
# cv2.dilate(thresh, dilate_kernel, thresh, iterations=2)
# # cv2.morphologyEx(thresh, cv2.MORPH_OPEN, erode_kernel) 
# # cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, erode_kernel)

# # Detect Contours in the thresholded image.

# contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# # contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)

# hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    
# cap.release()
# out.release()
# cv2.destroyAllWindows()

- [Link to Plantions Post](https://plantions.github.io/project/2020/06/30/sprint-5.html)

- [Shi Tomasi](https://docs.opencv.org/master/d4/d8c/tutorial_py_shi_tomasi.html)
- [Changing Video Resolution](https://theailearner.com/2018/11/15/changing-video-resolution-using-opencv-python/)
- [Niblack and Sauvola Threshholding](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_niblack_sauvola.html)

In [29]:
# @title # Orig Code Hide

# Clean Up Temporary Movie Files
try:
    os.remove("pls_delete.mp4")
    os.remove("pls_delete.mpeg")
    os.remove("pls_delete.wav")
    os.remove("pls_delete_play.mp4")
except FileNotFoundError:
    print("Downloading Videos...")
else:
    print("RM. Downloading Videos...")


#@title # Step 2: Video Processing

# Video Size Reducer
print("Starting time of video reduction: {}".format(datetime.now()))
start_time = time.time()

# Reduce video size to 480p
cap = cv2.VideoCapture('mimosa_400x.mov')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('reduced.avi',fourcc, 5, (640,480))
 
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(640,480),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()

# print("Video quality reduction to 480p took", round(((time.time() - start_time2)/60), 2), "minutes.")

# Create a VideoCapture object from Reduced Video Resolution 480p
cap = cv2.VideoCapture('reduced.avi')
arr = []
no_of_frame = 0
no_of_frames_to_process = (int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-4)
duration_of_video_s = (int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))/30
print('this is the duration: ', duration_of_video_s)

# Capture several frames to allow the camera's autoexposure to # adjust.
for i in range(3):
    success, frame = cap.read() 
if not success:
    exit(1)

# Default resolutions of the frame are obtained and casted to int.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create the KNN background subtractor.
bg_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows=True)
history_length = 5
bg_subtractor.setHistory(history_length)

erode_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 3))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
leafs = []
num_history_frames_populated = 0

fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
out = cv2.VideoWriter(OUTPUT_VIDEO_NAME_MPEG, fourcc, FRAMES_PER_SECOND, (frame_width, frame_height))

grabbed_frames = 0

# ---------DETECTION------------

while True:
    grabbed, frame = cap.read()
    grabbed_frames += 1
    no_of_frame = no_of_frame + 1
    if no_of_frame == 400:
        grabbed = False

    if (grabbed is False):
        break

    # Apply the KNN background subtractor.
    fg_mask = bg_subtractor.apply(frame)

    # Let the background subtractor build up a history.
    if num_history_frames_populated < history_length:
        num_history_frames_populated += 1
        continue

    # Create the thresholded image.
    
    # Thresholding
    # _, thresh = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
    thresh = cv2.adaptiveThreshold(fg_mask, 120, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    
    # Find intersting points
    # thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
    thresh = cv2.goodFeaturesToTrack(thresh,25,0.01,100)
    

    # for i in corners:
    #     x,y = i.ravel()
    #     cv2.circle(gray,(x,y),3,255,-1)
    # cv2_imshow(frame)

    # Morphology
    # cv2.erode(thresh, erode_kernel, thresh, iterations=2)
    # cv2.dilate(thresh, dilate_kernel, thresh, iterations=2)
    cv2.morphologyEx(thresh, cv2.MORPH_OPEN, erode_kernel) 
    # cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, erode_kernel)
    
    # Detect Contours in the thresholded image.
    
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Draw green rectangles around large contours.
    # Also, if no leafs are being tracked yet, create some.
    should_initialize_leafs = len(leafs) == 0
    id = 0
    print ("\r Processing {} of {} frames.".format(no_of_frame, no_of_frames_to_process), end="")
    # print('Number of frames to complete: ',no_of_frames_to_process)
    
# ---------TRACKING---------    

    for c in contours:
        if cv2.contourArea(c) > 100:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.circle(frame, (x, y), 3,
                            (0, 255, 0), 1)
            if should_initialize_leafs:
                leafs.append(Leaf(id, hsv_frame, (x, y, w, h)))
        frame_no_and_values = np.array([ id, x, y, w, h ])
        a = np.append(frame_no_and_values, [no_of_frame])
        arr.append(a)
        id += 1
        
    # Update the tracking of each leaf.
    for leaf in leafs:
        leaf.update(frame, hsv_frame)
        frame_no_and_values = np.array([ id, x, y, w, h ])
        a = np.append(frame_no_and_values, [no_of_frame])
        arr.append(a)
    out.write(frame)
    # Print Image every 3 seconds (i.e. modulo 90 frames)
    # if no_of_frame%4 == 0:
        # cv2_imshow(frame)
        # cv2_imshow(thresh)
        # cv2_imshow(hsv_frame)
        
    
cap.release()

out.release()
cv2.destroyAllWindows()
motion_array = np.array(arr)

print("\nProcessing took", round(((time.time() - start_time)/60), 2), "minutes.")

Starting time of video reduction: 2020-07-21 18:00:44.227797
this is the duration:  1.5666666666666667


error: ignored

In [19]:
### Movie Conversion from .mpeg to .mp4
start_time2 = time.time()
# Shortened videos to 15s to display them in Preview
subprocess.run('ffmpeg -i {} {} -loglevel quiet'.format(OUTPUT_VIDEO_NAME_MPEG, OUTPUT_VIDEO_NAME_MP4), shell=True)

# Shortening Duration of Video for Preview to max 15s or 
time_subvideo_to_be_converted = min(duration_of_video_s, 15)
round(time_subvideo_to_be_converted, 1) 
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip('pls_delete.mp4', 0, time_subvideo_to_be_converted, targetname="pls_delete_play.mp4",)
print("Conversion to mp4 and creating preview took", round(((time.time() - start_time2)/60), 2), "minutes.")


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i pls_delete.mp4 -ss 0.00 -t 1.57 -vcodec copy -acodec copy pls_delete_play.mp4
... command successful.
Conversion to mp4 and creating preview took 0.03 minutes.


In [20]:
# @title #Step 3: Preview the Video

def playvideo(filename):
    video = io.open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video width="80%" style="display:block; margin:0 auto;" alt="test" controls>
                    <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
                 </video>'''.format(encoded.decode('ascii')))
playvideo('pls_delete_play.mp4')


## Detection of Object 2

In [ ]:
import cv2

bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
erode_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

In [ ]:
cap = cv2.VideoCapture('paprika.mov')
for i in range(10):
    success, frame = cap.read()
if not success:
    exit(1)

In [ ]:
# gray_background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
# gray_background = cv2.GaussianBlur(gray_background, (BLUR_RADIUS, BLUR_RADIUS), 0)

In [ ]:
# success, frame = cap.read() 
# while success:
#     # gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
#     # gray_frame = cv2.GaussianBlur(gray_frame, (BLUR_RADIUS, BLUR_RADIUS), 0)
#     # diff = cv2.absdiff(gray_background, gray_frame)
#     fg_mask = bg_subtractor.apply(frame)
#     _, thresh = cv2.threshold(fg_mask, 244, 255, cv2.THRESH_BINARY)
#     cv2.erode(thresh, erode_kernel, thresh, iterations=2)
#     cv2.dilate(thresh, dilate_kernel, thresh, iterations=2)
#     contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     for c in contours:
#         if cv2.contourArea(c) > 500:
#             x, y, w, h = cv2.boundingRect(c)
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

#     # cv2_imshow(fg_mask) 
#     # cv2_imshow(thresh) 
#     # cv2_imshow(frame)

# k = cv2.waitKey(1) 
# if k == 27: # Escape
#     break

# success, frame = cap.read()


In [ ]:
# @title # Step 4: Data Visualizations

# ____ VIDEO PROCESSING ____
# This is the output NumPy Array from the video processing
data = motion_array

# Create a Pandas DataFrame from NumPy arrays with UniqueID, x- & y-Value of ROI (i.e., Region of Interest), as well as width and height of the accompanying moving windows. 
# Frame refers to video frame (30 fps) on which point was tracked.
# RESET DF
df_video = pd.DataFrame(data=data[0:,0:], index=[i for i in range(data.shape[0])], columns=['ID','X','Y','W','H','Frame'])

# Calculate Elapsed Time for Frames of Video
df_video["Elapsed"] = round(df_video["Frame"]/30, 1)

# Remove Duplicates
df_video_wo_dups = df_video.drop_duplicates(subset=None, keep='first', inplace=False)
df_video_queried = df_video_wo_dups.query('0 <= ID <= {}'.format(10))
# df_video_4_cols = df_video_queried.filter(items=['ID', 'X', 'Y', 'Elapsed'])
df_video_3_cols = df_video_queried.filter(items=['ID', 'Y', 'Elapsed'])

# Minding that Matplotlib inverts x/y Values
# df_inverted_x_y_minus_1 = df_video_4_cols.sub([0, frame_width, frame_height, 0], axis='columns')
df_inverted_x_y_minus_1 = df_video_3_cols.sub([0, frame_height, 0], axis='columns')
# df_inverted_x_y = df_inverted_x_y_minus_1.mul([1, -1, -1, 1], axis='columns')
df_inverted_x_y = df_inverted_x_y_minus_1.mul([1, -1, 1], axis='columns')
df_video = df_inverted_x_y

# Use Pivot Feature of Pandas DataFrame
# df_pivoted_video = pd.pivot_table(df_video, values=["X", "Y"], index=pd.Grouper(key='Elapsed'), columns=["ID"],)
df_pivoted_video = pd.pivot_table(df_video, values=["Y"], index=pd.Grouper(key='Elapsed'), columns=["ID"],)

# Create flexible number of indices based on number of Regions of Interest
int(df_video_queried.max(axis = 0)[0]); counter = 0
# x_indices = []
y_indices = []
while counter <= int(df_video_queried.max(axis = 0)[0]):
    # x_indices.append(('X' + str(counter))); 
    y_indices.append(('Y' + str(counter)))
    counter = counter + 1
df_pivoted_video.columns = y_indices # + x_indices

# ____AUDIO PROCESSING____

# Extracts Audio from Input Video
command = "ffmpeg -i {} -ab 160k -ac 2 -ar 44100 -vn {}".format(INPUT_VIDEO, OUTPUT_VIDEO_NAME_WAV)
subprocess.call(command, shell=True)

# Unpack Audio
audio_path = INPUT_VIDEO
x , sr = librosa.load(audio_path)

# Get the MFCCs (and choose the number of features to extract); manipulate Pandas DataFrame.
mfccs = librosa.feature.mfcc(x, sr=sr, n_mfcc=4)
audio_data = np.transpose(mfccs)

# Calculate the step length between a mfcc and the audio in seconds
window_length = 512 # default value by mfcc algorithm
length = (round((window_length/sr)*mfccs.shape[1]), 2)
step_size = length[0]/mfccs.shape[1]

# Getting Elapsed Time of Audio from Audiofile
audio_time_data = np.arange(0, (length[0]), step_size)
audio_time_data = np.round(audio_time_data, 1)
df_audio_time_steps = pd.DataFrame(data=audio_time_data, 
                index=[i for i in range(audio_time_data.shape[0])], 
                columns=['Elapsed_t'])
# Create the Audio Pandas DataFrame
df_audio = pd.DataFrame(data=audio_data, 
                index=[i for i in range(audio_data.shape[0])], 
                columns=['MFCC'+str(i) for i in range(audio_data.shape[1])])
df_audio_w_time = pd.concat([df_audio_time_steps, df_audio], axis=1, join='outer')
df_audio_w_time_w_o_dups = df_audio_w_time.drop_duplicates(subset='Elapsed_t', keep='first')
df_audio_w_time_w_o_dups

## Merge AUDIO and VIDEO
# df_merged = pd.concat([df_audio_w_time_w_o_dups, df_pivoted_video], axis = 1, join= 'outer')
df_merged = pd.merge(df_audio_w_time_w_o_dups, df_pivoted_video, how='left', left_on=['Elapsed_t'], right_on=['Elapsed']) # add 'indicator=True' to see functionality
df_bf_filled = df_merged.fillna(method='bfill') # backward filling (NANs filling up)
df_filled = df_bf_filled.fillna(method='ffill') # forward filling (NANs filling down)

# Melt from wide to long-form Data Structure
df_filled_every_two_seconds = df_filled.iloc[::20]
df_filled_melted = df_filled_every_two_seconds.melt('Elapsed_t', var_name='Variables', value_name='Values')

alt.Chart(df_filled_melted).mark_line().encode(
  x='Elapsed_t',
  y='Values',
  color='Variables'
  ).properties(
    width=900,
    height=400,
).interactive(bind_y=False)

In [ ]:
# @title # Step 5: Happiness Indicator

total_happiness_counter = 0
per_curve_happiness_counter = 0
img_happy = cv2.imread('happy.png', cv2.IMREAD_UNCHANGED)
img_sad = cv2.imread('sad.png', cv2.IMREAD_UNCHANGED)

# obviously one cannot assume that the height is fixed
HEIGHT = 100
# HEIGHT = (max(x[peaks])/2 )

# To Do: has to be a regex and start with y(i) values
for tracked_move in df_filled.columns[5:]:
    x = df_filled[tracked_move]
    # try without height bc its an absolute value
    peaks, _ = find_peaks(x, HEIGHT, distance=50)
    per_curve_happiness_counter = 0
    per_curve_happiness_indicators = len(x[peaks])
    happiness_threshold = (max(x[peaks]) + min(x[peaks]))/2
    
    for value in x[peaks]:
        if value > happiness_threshold:
            per_curve_happiness_counter = per_curve_happiness_counter + 1
            total_happiness_counter = total_happiness_counter + 1

if per_curve_happiness_counter > (per_curve_happiness_indicators/2):
    print("This plant observed that the speaker is >>> happy <<<.")
    # cv2_imshow(img_happy)
else:
    print("This plant observed that the speaker is >>> sad <<<.")
    # cv2_imshow(img_sad)

plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.show()    

# Additional Features

In [ ]:
# @title ## Dummy Sad Plant

img_happy = cv2.imread('happy.png', cv2.IMREAD_UNCHANGED)
img_sad = cv2.imread('sad.png', cv2.IMREAD_UNCHANGED)
print("This plant observed that the speaker is >>> sad <<<.")
cv2_imshow(img_sad)

In [ ]:
# @title ## Camera Capturing [in Process]

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
# @title ## Export to Excel
df_filled.to_excel('11_min_video.xlsx')
df_filled_melted.to_excel('11_min_video_melted.xlsx')

In [ ]:
# @title ## Interactive Scatter Plot

# df_filled

# alt.Chart(df_filled_melted).mark_circle().encode(
#     alt.X(alt.repeat("column"), type='quantitative'),
#     alt.Y(alt.repeat("row"), type='quantitative'),
# ).properties(
#     width=150,
#     height=150
# ).repeat(
#     row=['Elapsed_t', 'Variables', 'Values'],
#     column=['Variables', 'Elapsed_t', 'Values']
# ).interactive()

In [ ]:
# @title ## Check for Movie Duration
import moviepy.editor

# Converts into more readable format
def convert(seconds):
    hours = seconds // 3600
    seconds %= 3600

    mins = seconds // 60
    seconds %= 60

    return hours, mins, seconds

# Duration of Input Video
video = moviepy.editor.VideoFileClip(INPUT_VIDEO)
video_duration = float(video.duration)
hours, mins, secs = convert(video_duration)
print("The Video named {} Seconds:".format(INPUT_VIDEO), secs)

# Duration of Resulting Video
video2 = moviepy.editor.VideoFileClip(OUTPUT_VIDEO_NAME_MP4)
video_duration2 = float(video2.duration)
hours, mins, secs = convert(video_duration2)
print("The Video named {} Seconds:".format(OUTPUT_VIDEO_NAME_MP4), secs)

# Cross-check Length of Video and Audio
print("Length of Audio in Seconds: ", (512/sr)*mfccs.shape[1])

In [ ]:
# @title ## Getting FPS of Video

#!/usr/bin/env python

import cv2
import time

if __name__ == '__main__' :

    # Start default camera
    video = cv2.VideoCapture(INPUT_VIDEO)

    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

    # Release video
    video.release()

In [ ]:
# @title ## File Uploader
# Imports
from google.colab import files
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display
from IPython.display import clear_output
from datetime import datetime
import cv2
import numpy as np
import pandas as pd
import subprocess
import io
import os
import base64
import librosa
import librosa.display
import altair as alt
from scipy.signal import chirp, find_peaks, peak_widths
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
!curl -o happy.png -L https://i.imgur.com/PXpWO5Cs.png -s
!curl -o sad.png -L https://i.imgur.com/mAYh4Qts.png -s
uploaded = files.upload()
for fn in uploaded.keys():
  print('Successfully uploaded file "{NAME_OF_UPLOADED_VIDEO}".'.format(
      NAME_OF_UPLOADED_VIDEO=fn,))


In [ ]:
# @title ## Audio File Player

# Needed imports
from IPython.display import Audio

audio_name = 'test2.mp3'

# Generate a player for mono sound
Audio(audio_name,autoplay=True)

Please send Feedback to sduerr@mit.edu.